# 将CSV文件数据存储到本地SQLite数据库

本Notebook演示如何将`GraduateEmploymentSurvey.csv`文件的数据存储到本地SQLite数据库（.db文件），并为表添加自增主键id。

## 1. 导入所需库
导入pandas和sqlite3等必要的Python库。

In [6]:
import pandas as pd
import sqlite3
import os

## 2. 读取CSV文件
使用pandas的read_csv函数读取本地CSV文件。

In [7]:
csv_path = 'GraduateEmploymentSurvey.csv'
df = pd.read_csv(csv_path)

## 3. 预览CSV数据
显示CSV文件的前几行，确认数据格式和列名。

In [8]:
df.head()

,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile
0,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy and Business,97.4,96.1,3701,3200,3727,3350,2900,4000
1,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy (3-yr direct Honours Programme),97.1,95.7,2850,2700,2938,2700,2700,2900
2,2013,Nanyang Technological University,College of Business (Nanyang Business School),Business (3-yr direct Honours Programme),90.9,85.7,3053,3000,3214,3000,2700,3500
3,2013,Nanyang Technological University,College of Business (Nanyang Business School),Business and Computing,87.5,87.5,3557,3400,3615,3400,3000,4100
4,2013,Nanyang Technological University,College of Engineering,Aerospace Engineering,95.3,95.3,3494,3500,3536,3500,3100,3816


## 4. 创建SQLite数据库并建立表结构
使用sqlite3连接本地数据库文件，创建表结构，字段与CSV列一致，并添加自增主键id。

In [9]:
db_path = 'GraduateEmploymentSurvey.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 获取列名和类型，全部用TEXT类型
columns = df.columns.tolist()
columns_def = ', '.join([f'"{col}" TEXT' for col in columns])

# 创建表，添加自增主键id
create_table_sql = f'''
CREATE TABLE IF NOT EXISTS survey (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    {columns_def}
);
'''
cursor.execute(create_table_sql)
conn.commit()

## 5. 将CSV数据写入SQLite数据库
遍历DataFrame，将数据插入到SQLite表中。

In [11]:
# 插入数据（修正 f-string 语法）
# 构建列名列表字符串
cols_list = ', '.join([f'"{col}"' for col in columns])
placeholders = ', '.join(['?'] * len(columns))
insert_sql = f'INSERT INTO survey ({cols_list}) VALUES ({placeholders})'

print(f'插入 SQL 语句: {insert_sql[:100]}...')
print(f'准备插入 {len(df)} 行数据...')

# 将 NaN 转为 None 便于 SQLite 存储
values = df.where(pd.notnull(df), None).values.tolist()
cursor.executemany(insert_sql, values)
conn.commit()

# 验证插入结果
count = cursor.execute('SELECT COUNT(*) FROM survey').fetchone()[0]
print(f'✓ 成功插入 {count} 行数据到表 survey')

插入 SQL 语句: INSERT INTO survey ("year", "university", "school", "degree", "employment_rate_overall", "employment...
准备插入 1401 行数据...
✓ 成功插入 1401 行数据到表 survey


## 6. 验证数据写入结果
从数据库中查询部分数据，确认写入成功。

In [12]:
# 查询前5条数据
result = pd.read_sql_query('SELECT * FROM survey LIMIT 5', conn)
result

,id,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile
0,1,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy and Business,97.4,96.1,3701,3200,3727,3350,2900,4000
1,2,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy (3-yr direct Honours Programme),97.1,95.7,2850,2700,2938,2700,2700,2900
2,3,2013,Nanyang Technological University,College of Business (Nanyang Business School),Business (3-yr direct Honours Programme),90.9,85.7,3053,3000,3214,3000,2700,3500
3,4,2013,Nanyang Technological University,College of Business (Nanyang Business School),Business and Computing,87.5,87.5,3557,3400,3615,3400,3000,4100
4,5,2013,Nanyang Technological University,College of Engineering,Aerospace Engineering,95.3,95.3,3494,3500,3536,3500,3100,3816
